In [238]:
!pip install pandas
!pip install beautifulsoup4
!pip install twilio

from twilio.rest import Client
import pandas as pd
from datetime import datetime
import numpy as np
import bs4 as bs
import requests

In [223]:
wiki_link = 'https://en.wikipedia.org'
weblink = wiki_link + '/wiki/List_of_UFC_events'
response = requests.get(weblink)
soup = bs.BeautifulSoup(response.text, 'lxml')
table = soup.find_all('table', {'class':'wikitable'})[0]

In [224]:
ufc_events = pd.read_html(str(table))[0]
ufc_events.drop(columns='Ref.', inplace=True)
ufc_events.rename(columns={'Unnamed: 2':'Venue'}, inplace=True)

In [225]:
#add link to fight card

links=[]

for row in table.find_all('tr')[1:]:
    link = row.find_all('td')[0].contents[0]
    links.append(wiki_link + link.get('href'))
    
ufc_events['Fight Card Link'] = links

In [226]:
#fix date format

for index, row in ufc_events.iterrows():
    old_date = row['Date']
    new_date = datetime.strptime(old_date, '%b %d, %Y')
    row['Date'] = new_date.strftime('%d %b %Y')

In [243]:
ufc_events.tail(1)

,Event,Date,Venue,Location,Fight Card Link
25,UFC Fight Night: Gustafsson vs. Smith,01 Jun 2019,Ericsson Globe,"Stockholm, Sweden",https://en.wikipedia.org/wiki/UFC_Fight_Night:...


In [268]:
next_ufc_event = ufc_events.tail(1)
message_body = str(next_ufc_event.iloc[0,0]) + "\n" + str(next_ufc_event.iloc[0,3]) + "\n" + str(next_ufc_event.iloc[0,1]) + "\n" + str(next_ufc_event.iloc[0,4])
print(message_body)

UFC Fight Night: Gustafsson vs. Smith
Stockholm, Sweden
01 Jun 2019
https://en.wikipedia.org/wiki/UFC_Fight_Night:_Gustafsson_vs._Smith


In [269]:
account_sid = 'ACc2c5c19fd903a29de3492a7f1f110378'
auth_token = 'd8d3d91d148c07cfc5c4561b0286131d'
client = Client(account_sid, auth_token)

ed = 'whatsapp:+447771580615'
jay = 'whatsapp:+447749956110'

In [270]:
def send_whatsapp(recipient, message_body):
    message = client.messages.create(
                                  body= message_body,
                                  from_= 'whatsapp:+14155238886',
                                  to= recipient
                              )

    print(message.sid)

In [271]:
send_whatsapp(jay, message_body)

SMc74e3385c212403a858b4779dc70fe3a
